In [18]:
import pandas as pd
import datetime
import re
import numpy as np
import numpy.polynomial.polynomial as poly
import sys, os
from math import pi
import re

from scipy.optimize import curve_fit

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from numpy import inf
np.random.seed(123)
from sklearn.metrics import r2_score

from bokeh.plotting import output_notebook, figure, show
from bokeh.palettes import BuPu9, GnBu9, Category20c, RdYlBu, OrRd, RdBu
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, BoxSelectTool, Range1d, Rect, LabelSet, BooleanFilter, BoxAnnotation, DatetimeTickFormatter, CDSView, GroupFilter, NumeralTickFormatter, Label
from bokeh.layouts import row
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.transform import linear_cmap

from bokeh.plotting import figure
from bokeh.transform import cumsum

output_notebook()

#Plotting options
opts = dict(width=1200, height=600, toolbar_location="above",
      tools='tap, box_zoom, pan, undo, crosshair, reset, wheel_zoom, box_select, save')


Loading BokehJS ...

In [14]:
#Return list of cases per state and county
def populateCases(state, county, population=None):
    state = str(state)
    county = str(county)
    listToReturn = []
    
    for date in dfDates:
        tempValue = df[ (df['Province_State'] == state) & (df['Admin2'] == county) ][date].tolist()[0]
        
        if population == None:
            listToReturn.append(int(tempValue))
        else:
            listToReturn.append(float(tempValue/population))
            
    #print(state, county, listToReturn)
    return listToReturn

#Return Population per state and county
def getPopulation(state, county):
    #New York is split into two counties. Sum them. Populate cases is for NYC
    if (state == 'New York') & (county == 'New York'):
        popSeries = dfPopulate['POPESTIMATE2019'][ (dfPopulate['STNAME'] == state) & (dfPopulate['CTYNAME'] == 'Bronx County') ]
        print(popSeries)
        pop = popSeries.tolist()[0]
        popSeries = dfPopulate['POPESTIMATE2019'][ (dfPopulate['STNAME'] == state) & (dfPopulate['CTYNAME'] == 'Kings County') ]
        print(popSeries)
        pop = pop + popSeries.tolist()[0]
        pop = 8600000000
    else:
        county = county + ' County'
        popSeries = dfPopulate['POPESTIMATE2019'][ (dfPopulate['STNAME'] == state) & (dfPopulate['CTYNAME'] == county) ]
        pop = popSeries.tolist()[0]
        #print(pop)
    return pop

#For curve fitting
def exponential(x, a, k):
    return a*np.exp(x*k)

#Seeing how well the fit fits
def rms(y, yfit):
    return np.sqrt(np.sum((y-yfit)**2))

In [15]:
#Covid Data
gitFile = 'COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

#Population Data
popFile = 'co-est2019-alldata.csv'

#States and Counties to get data for
dataToGrab = [['Florida', 'Palm Beach'], ['Texas', 'Dallas'], ['New York', 'New York'], 
              #['California', 'Los Angeles'], ['Georgia', 'Fulton'], ['Georgia', 'DeKalb'],
              ['Florida', 'Miami-Dade'], 
              ['Massachusetts', 'Suffolk'],
              ['New Jersey', 'Essex'],
              ['Tennessee', 'Davidson'], ['Tennessee', 'Hamilton'], ['Tennessee', 'Knox'], ['Kentucky', 'Fayette'],
             ]

#Divide by population - T/F
divideByPopulation = False

#Read CSVs
df = pd.read_csv(gitFile)
dfPopulate = pd.read_csv(popFile, engine='python')

#Append population
for pop in dataToGrab:
    population = getPopulation(pop[0], pop[1])
    #Add population to list
    pop.append(population)
    
#Dates are embedded into the columns - Ignore first 11 columns
dfDates = list(df.columns)[11:] 

#Create empty list for storing multiple counties
countyStateCases = {}
for itm in dataToGrab:
    #Get cases per location
    if divideByPopulation:
        tempData = populateCases(itm[0], itm[1], itm[2])
    else:
        tempData = populateCases(itm[0], itm[1])
    #Combine county+_+state for key
    key = str(itm[0])+ '_' +str(itm[1])
    countyStateCases[ key ] = tempData

#Get dates as datetime
dataDates = [datetime.datetime.strptime(x, '%m/%d/%y') for x in dfDates]

#Append dates to dict
countyStateCases[ 'dates' ] = dataDates

#Convert to DF
dfToPlot = pd.DataFrame.from_dict( countyStateCases )
#dfToPlot

1863    1418207
Name: POPESTIMATE2019, dtype: int64
1884    2559903
Name: POPESTIMATE2019, dtype: int64


In [26]:
#Plot data
pltA = figure(**opts, x_axis_type="datetime")

#Column names are the counties to plot
columnCounties = dfToPlot.columns.tolist()
columnCounties.remove('dates');

#Gets dates
columnCountiesDates = dfToPlot['dates']

#Add data from dfToPlot
for idx, location in enumerate(columnCounties, start=0):
    #Parse Name
    legendName = location.replace('_', ', ')
    #Get color
    lineColor = str(RdBu[len(columnCounties)][idx])
    
    #Add line
    pltA.line( x=dfToPlot['dates'], y=dfToPlot[location], color=lineColor, legend_label=legendName, alpha=0.5, line_width=2)
    pltA.scatter( x=dfToPlot['dates'], y=dfToPlot[location], color=lineColor, legend_label=legendName, alpha=0.5, line_width=2)

    #Get xAxis in list 0...X
    xData = np.arange(len(columnCountiesDates), dtype=np.float64)
    
    #Get data and convert 
    yData = dfToPlot[location].tolist()
    yData = np.asarray(yData, dtype=np.float64 )
    
    #Curve fit exponential
    popt, pcov = curve_fit(exponential, xData,  yData, p0=(4, 0.1) )
    yFit = exponential(xData, *popt)
    
    #Curve fit polynominal
    coefs = poly.polyfit(xData, yData, 5)
    ffit = poly.polyval(xData, coefs)
    print( r2_score(yData, ffit) )

    #plot exponential
    #pltA.line(dfToPlot['dates'], yFit, line_dash='dashed', color='black', legend_label=legendName)
    
    #Plot poly
    pltA.line(dfToPlot['dates'], ffit, line_dash='dotted', color='white', legend_label=legendName)
    
    #How well does it fit
    #print('r2 error for {} fit: {:0.4f}'.format( location.replace('_', ', '), r2_score(yData, yFit) ))

    
#Update location of the legend
pltA.legend.location = 'top_center'
pltA.legend.click_policy="hide"

#Update Title
if divideByPopulation:
    pltA.title.text = 'Cases in Select US Counties divided by Population'
    pltA.yaxis.axis_label = 'Number of Cases Divided by Population'
else:
    pltA.title.text = 'Cases in Select US Counties'
    pltA.yaxis.axis_label = 'Number of Cases'

#Add Hatch Pattern
pltA.xgrid.band_hatch_pattern = "\\"
pltA.xgrid.band_hatch_alpha = 0.4
pltA.xgrid.band_hatch_color = "lightgrey"
pltA.xgrid.band_hatch_weight = 0.4
pltA.xgrid.band_hatch_scale = 20

#Format to always show Month/Day
pltA.xaxis.formatter=DatetimeTickFormatter(days="%m/%d",
    months="%m/%d",
    hours="%m/%d",
    minutes="%m/%d")

#Add more ticks!
pltA.xaxis.ticker.desired_num_ticks = int(len(dfToPlot)/2)

pltA.background_fill_color = "#adadad"

show(pltA)

0.9875622058980708
0.9915525197099062
0.9919501597134602
0.9888519366597923
0.9959492950375503
0.9912242136797211
0.9844322202462843
0.9759385523969354
0.9723377634446112
0.9666980493759083
